# Day 09. Exercise 02
# Metrics

## 0. Imports

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
from tqdm.notebook import tqdm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
df.head()

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [11]:
def calc_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', average='weighted')

    print(f"accuracy is {accuracy:.5f}")
    print(f"precision is {precision:.5f}")
    print(f"recall is {recall:.5f}")
    print(f"roc_auc is {roc_auc:.5f}")

In [12]:
# Best parameters: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}
best_params = {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}

svm_model = SVC(**best_params)
svm_model.fit(X_train, y_train)

calc_metrics(svm_model, X_test, y_test)

accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97910


## 3. Decision tree

1. The same task for decision tree

In [13]:
# Best parameters: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}
tree_best_params = {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}

dt_model = DecisionTreeClassifier(**tree_best_params)

dt_model.fit(X_train, y_train)

calc_metrics(dt_model, X_test, y_test)

accuracy is 0.89349
precision is 0.89536
recall is 0.89349
roc_auc is 0.93806


## 4. Random forest

1. The same task for random forest.

In [14]:
# Best parameters: {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50}
rf_best_params={'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50}

rf = RandomForestClassifier(**rf_best_params)
rf.fit(X_train, y_train)

calc_metrics(rf, X_test, y_test)

accuracy is 0.93787
precision is 0.93967
recall is 0.93787
roc_auc is 0.98713


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

Best RandomForest

In [20]:
all = pd.DataFrame(y_test.value_counts()).sort_values(by='dayofweek')
err = pd.DataFrame(y_test[y_test != rf.predict(X_test)].value_counts()).sort_values(by='dayofweek')
(err/all*100).sort_values(by='count', ascending=False)

,count
dayofweek,
0,18.518519
4,14.285714
1,9.090909
2,6.666667
5,5.555556
3,2.500000
6,1.408451


In [23]:
pd.DataFrame(X_test[y_test != rf.predict(X_test)])[['numTrials']].value_counts()

numTrials
1            8
5            3
2            2
8            2
16           1
19           1
29           1
54           1
75           1
84           1
Name: count, dtype: int64

In [35]:
filename = 'rf_model.pkl'
joblib.dump(rf, filename)

['rf_model.pkl']

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [32]:
def evaluate_models(models, params_list, X_train, y_train, X_test, y_test):
    results = {}
    for model, params in zip(models, params_list):
        model_instance = model(**params)
        
        model_instance.fit(X_train, y_train)
        
        y_pred = model_instance.predict(X_test)
        y_pred_proba = model_instance.predict_proba(X_test)

        accuracy = accuracy_score(y_test, y_pred)

        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')

        roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', average='weighted')
        
        results[str(model_instance)] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'roc_auc': roc_auc
        }
    
    return results

In [34]:
models = [RandomForestClassifier, SVC, DecisionTreeClassifier]
params_list = [
    {'n_estimators': 50, 'max_depth': 10, 'class_weight': 'balanced', 'criterion': 'entropy'},
    {'C': 1.0, 'kernel': 'linear', 'probability': True},
    {'criterion': 'entropy', 'max_depth': 10, 'class_weight': 'balanced'}
]

results = evaluate_models(models, params_list, X_train, y_train, X_test, y_test)

for model, metrics in results.items():
    print('----------------------------------------------------------------')
    print(f"{model}:")
    for metric, value in metrics.items():
        print(f" {metric}: {value:.5f}")

----------------------------------------------------------------
RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=10, n_estimators=50):
 accuracy: 0.89645
 precision: 0.90242
 recall: 0.89645
 roc_auc: 0.97590
----------------------------------------------------------------
SVC(kernel='linear', probability=True):
 accuracy: 0.71893
 precision: 0.72714
 recall: 0.71893
 roc_auc: 0.91248
----------------------------------------------------------------
DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=10):
 accuracy: 0.79882
 precision: 0.81564
 recall: 0.79882
 roc_auc: 0.93589
